In [1]:
import librosa
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import os
from scipy.signal import butter, lfilter
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import relu, tanh, linear
from tensorflow.keras.utils import Progbar
from joblib import Parallel, delayed #Paralleize calculation
import psycopg2
from sqlalchemy import create_engine, Column, Integer, ARRAY, MetaData, Table, Text
from sqlalchemy.dialects.postgresql import ARRAY as PG_ARRAY
from psycopg2.extensions import register_adapter, AsIs
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# check if tensorflow uses GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-01-04 15:21:34.798559: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 15:21:34.798590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 15:21:34.799583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 15:21:34.805539: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 15:21:35.549391: W tensorflow/compiler/tf2

Num GPUs Available:  1


In [2]:
# Custom adapter function for postgre
def adapt_numpy_ndarray(numpy_array):
    return AsIs(list(numpy_array))
# Register the postgre-adapter
register_adapter(np.ndarray, adapt_numpy_ndarray)

# Database connection parameters and alchemy engine
dbname = 'bathunting'
user = 'python'
password = 'python_password'
host = 'localhost'
port = '5432' 
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Functions to get data
def get_target_data(target, limit=0, no_target=False):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    #query = ""
    if no_target:
        query = f"SELECT new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    else:
        query = f"SELECT target, new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    df = pd.read_sql_query(query, engine)
    if no_target:
        df = pd.DataFrame(df['new_arr'].tolist())
    return df

# messy, i know
def get_targets_to_data(limit=0):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    query = f"SELECT target FROM batcall where 10 < ANY(new_arr) {lmt}"
    df = pd.read_sql_query(query, engine)
    return df

def get_all_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    return all_df

def get_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    df = pd.concat(all_df)
    return df

# try to visualize only the maximum values per species
def spectrogram_range(target):
    df = get_target_data(target, limit=0, no_target=True)
    max_vals = df.max()
    min_vals = df.min()
    
    abs_pos = max_vals.abs()
    abs_neg = min_vals.abs()
    mask = (abs_pos > abs_neg).astype(bool)
    return min_vals.where(mask, max_vals), max_vals.where(mask, min_vals)

def get_targets():
    #conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    #cursor = conn.cursor()
    query = f"SELECT target, bat FROM batcall group by target, bat order by target"
    df = pd.read_sql_query(query, engine)
    #conn.close()
    return df

def get_shape(nested_list):
    try:
        # Initialize shape list
        shape = []

        # Iterate to calculate the shape
        while isinstance(nested_list, list) or isinstance(nested_list, np.ndarray):
            shape.append(len(nested_list))
            nested_list = nested_list[0]

        return tuple(shape)
    except (TypeError, IndexError) as e:
        # In case the nested lists are not uniformly sized
        return f"Irregular shape - nested lists are not of equal size. \n ERROR: {e}"
    # Get data to work with
def get_features_and_targets(limit=100, scaler=StandardScaler()):
    data = get_data(limit=limit)

    df = pd.DataFrame(data["new_arr"].tolist())
    if scaler != None:
        df = scaler.fit_transform(df)

    labels = pd.DataFrame(data["target"])
    labels = to_categorical(labels, num_classes=19)
    return df, labels

In [27]:
# Checking the shape
df, labels = get_features_and_targets(limit=100, scaler=None) #limit=1000 is zu viel
print(labels.shape)
print(df.shape)

(1900, 19)
(1900, 4410)


In [28]:
# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)
#X_train = X_train[:-13]
#y_train = y_train[:-13]
print(X_train.shape)
print(y_train.shape)

(1520, 4410)
(1520, 19)


In [29]:
# Determine the number of input features
input_dim = X_train.shape[1]

# Build the model
model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dense(19, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [30]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=20)  # Adjust epochs and batch_size as needed

2024-01-04 15:48:14.338433: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 51.14MiB (rounded to 53625600)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-04 15:48:14.338484: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-01-04 15:48:14.338505: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 22, Chunks in use: 21. 5.5KiB allocated for chunks. 5.2KiB in use in bin. 420B client-requested in use in bin.
2024-01-04 15:48:14.338520: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 3, Chunks in use: 3. 2.2KiB allocated for chunks. 2.2KiB in use in bin. 2.2KiB client-requested in use in bin.
2024-01-04 15:48:14.338533: 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

38932: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] Free  at 7f5116082300 of size 1280 next 27
2024-01-04 15:48:14.338936: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116082800 of size 768 next 28
2024-01-04 15:48:14.338945: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116082b00 of size 256 next 33
2024-01-04 15:48:14.338949: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116082c00 of size 256 next 30
2024-01-04 15:48:14.338952: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116082d00 of size 256 next 34
2024-01-04 15:48:14.338956: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] Free  at 7f5116082e00 of size 256 next 35
2024-01-04 15:48:14.338959: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116082f00 of size 768 next 36
2024-01-04 15:48:14.338963: I external/local_tsl/tsl/framework/bfc_allocator.cc:1095] InUse at 7f5116083200 of size 172288 n

In [25]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (accuracy*100))

321/321 [==============================] - 1s 1ms/step - loss: 2.1865 - accuracy: 0.4103
Test Accuracy: 41.03


In [38]:
data_reshaped = []
for _,data in df.iterrows():
    # Normalize
    data -= np.mean(data)
    data /= np.std(data)

    # Realy no idea just assuming prof did it right
    # Calculate spectrogram with FFT
    stft = np.abs(librosa.stft(np.array(data), n_fft=512, hop_length=32))
    stft = 10 * np.log10(stft)
    stft = np.nan_to_num(stft)
    # Scale between [0,1] and reduce shape if needed
    stft = (stft - np.min(stft)) / (np.max(stft) - np.min(stft))
    stft = np.reshape(stft, (257, 138, 1))
    #stft = stft[:256, -128: , :]
    data_reshaped.append(stft)
df_reshaped = np.array(data_reshaped)

/tmp/ipykernel_2713/2456465814.py:10: RuntimeWarning: divide by zero encountered in log10
  stft = 10 * np.log10(stft)


In [ ]:
# Build the model
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_dim))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(19, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

In [51]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (accuracy*100))

57/57 [==============================] - 0s 5ms/step - loss: 2.9402 - accuracy: 0.0419
Test Accuracy: 4.19


 alles Mist!